# Building the Movie Maker Extended Skill
This notebook demonstrates how to use the Cortex Python SDK to build a simple Skill.  For simple cases, the entire Skill can be defined, tested, and deployed right from a notebook.

## Define the movie_maker_extended function


The movie_maker_extended action differs from the movie_maker in terms of it's inputs and outputs

Inputs:

* role: The role in which to replace ('lead_1', 'lead_2', 'support_1', 'support_2')

* new_cast: The name of the new cast member

* name_of_movie: The name of the fake movie which will be used to generate the 'movie' object containing the fake cast

Outputs:

* role: The role in which to replace ('lead_1', 'lead_2', 'support_1', 'support_2')

* new_cast: The name of the new cast member

* movie: The fake movie details generated by the 'movie_maker_extended' skill

These outputs will be used as input to the replace_cast skill which will replace the given role with the new_cast.

## Create *Extended Movie Maker* Daemon

In [ ]:
#Run this cell if your token/session has expired.

from cortex import Cortex
Cortex.login()

In [ ]:
# Common Setup
%run setup.ipynb

## Build and Deploy Action

In [ ]:
%%cortex_action --name '<YOURNAME>/<movie_maker_extended_YOUR_INITIALS>' --function movie_maker_extended --requirements faker --daemon

from faker import Faker
from cortex import Cortex, Message

def movie_maker_extended(params):
    msg = Message(params)
    cortex = Cortex.client(api_endpoint=msg.apiEndpoint, token=msg.token)
    fake = Faker()
    role = msg.payload.get('role')
    new_cast = msg.payload.get('new_cast')
    name_of_movie = msg.payload.get('name_of_movie')
    movie = {'name_of_movie': name_of_movie, 'lead_1': fake.name(), 'lead_2': fake.name(), 'support_1': fake.name(), 'support_2': fake.name()}
    return cortex.message({'movie': movie, 'role': role, 'new_cast': new_cast}).to_params()

### Testing Actions
Using the Cortex client, we can test our Action to make sure it deployed properly.

In [ ]:
# Instantiate Cortex Client
cortex = Cortex.client()

# Retrieve our Action that was deployed above
action = cortex.action('<YOURNAME>/<movie_maker_extended_YOUR_INITIALS>')

The Action deployment status should say **COMPLETED**.  This will indicate that our Action is ready to invoke.

In [ ]:
action.get_deployment_status()

Invoke the Action using a Message object.  Here we just pass in the expected _num_ parameter in the Message payload.

In [ ]:
from cortex import Message
rs = action.invoke(cortex.message({'name_of_movie': "Our Big Adventure", 'role': 'lead_2', 'new_cast': 'Darrell Culbertson'}))
rs.payload

## Building a Cortex Skill
Now that our Action is ready and tested, we can move on to building a Cortex Skill.  In this simple example, our Skill will just pipe an Input to our Action and route the Output back to the caller.

In [ ]:
builder = cortex.builder()

In [ ]:
b = builder.skill('<YOURNAME>/movie_maker_extended-<YOUR INITIALS>').title('Movie Maker Extended-<YOUR_INITIALS>').description('Skill to generate a fake cast for a movie Part 2')

Next, we use the Input sub-builder to construct our Skill Input.  This is where we declare how our Input will route messages.  In this simple case, we use the _all_ routing which routes all input messages to same Action for processing and declares which Output to route Action outputs to.  We pass in our Action from the previous section to wire the Skill to the Action (we could have also passed in the Action name here).  Calling _build_ on the Input will create the input object, add it to the Skill builder, and return the Skill builder.

In [ ]:
b = b.input('make-movie').title('Name of Movie').parameter(name='name_of_movie', type='string').parameter(name='role', type='string', required=False).parameter(name='new_cast', type='string', required=False).all_routing(action, 'movie').build()

In the previous step, we referenced an Output called **greeting**.  We can create that Output here using the Output sub-builder.

In [ ]:
b = b.output('movie').title('Movie').parameter(name='movie', type='object', required=True).parameter(name='role', type='string', required=False).parameter(name='new_cast', type='string', required=False).build()

## Preview the CAMEL Document
We can preview the CAMEL document that each builder will create using the _to__camel_ method.

In [ ]:
b.to_camel()

## Final Build and Publish
We are now ready to build and publish our Skill.

In [ ]:
skill = b.build()
print('%s (%s) v%d' % (skill.title, skill.name, skill.version))

In [ ]:
rs = skill.invoke(input_name='make-movie', message=cortex.message({'name_of_movie': "Our Big Adventure", 'role': 'lead_2', 'new_cast': 'Jimmy Falon'}))
rs.payload